In [1]:
#import relevant libraries 
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv(r"C:\Users\USER\Downloads\311_Resolution_Satisfaction_Survey (1).csv")
data.head()

,Unique Key,Agency Acronym,Agency Name,Complaint Type,Descriptor,Borough,Resolution Description,Survey Year,Survey Month,Satisfaction Response,Dissatisfaction Reason
0,16657800-00015172,DOB,Department of Buildings,Adult Establishment,Zoning Violation,MANHATTAN,The Department of Buildings investigated this ...,2022,10,Strongly Agree,NaN
1,58524985-00281320,DOB,Department of Buildings,Adult Establishment,Zoning Violation,BROOKLYN,The Department of Buildings investigated this ...,2024,11,Strongly Disagree,The Agency did not correct the issue.
2,75073480-00178242,DFTA,Department for the Aging,Legal Services Provider Complaint,NaN,MANHATTAN,The Department for the Aging contacted you and...,2024,3,Strongly Agree,NaN
3,46435198-00170848,DOB,Department of Buildings,Advertising Sign,Poster,MANHATTAN,The Department of Buildings reviewed this comp...,2024,2,Neutral,NaN
4,40782130-00128055,DOB,Department of Buildings,Advertising Sign,Billboard,MANHATTAN,The Department of Buildings investigated this ...,2023,10,Strongly Disagree,"Status updates were unhelpful, inaccurate, inc..."


In [3]:
data.columns

Index(['Unique Key', 'Agency Acronym', 'Agency Name', 'Complaint Type',
       'Descriptor', 'Borough', 'Resolution Description', 'Survey Year',
       'Survey Month', 'Satisfaction Response', 'Dissatisfaction Reason'],
      dtype='object')

In [4]:
from transformers import pipeline
import torch
import warnings

warnings.filterwarnings("ignore")

In [8]:
from transformers import pipeline
import torch
import pandas as pd

# Load the T5 pipeline
generator = pipeline("text2text-generation", model="t5-base", tokenizer="t5-base")

# Example DataFrame
data = {
    "Agency Name": ["Department of Buildings"],
    "Complaint Type": ["Adult Establishment"],
    "Descriptor": ["Zoning Violation"],
    " Resolution Description": ["The Department of Buildings investigated this complaint and determined that no further action was necessary."],
    "Dissatisfaction Reason": ["Agency did not correct the issue"],
}
df = pd.DataFrame(data)

# Create prompt
def create_prompt(row):
    return (
        f"Generate a detailed justified reason for this citizen complaint:\n"
        f"Agency: {row['Agency Name']}\n"
        f"Complaint Type: {row['Complaint Type']}\n"
        f"Descriptor: {row['Descriptor']}\n"
        f"Dissatisfaction Reason: {row['Dissatisfaction Reason']}\n"
        f"Resolution Description: {row[' Resolution Description']}\n"
        f"Justified Reason:"
    )

# Apply prompt to DataFrame
df["input_text"] = df.apply(create_prompt, axis=1)

# Generate output
df["Justified Reason"] = df["input_text"].apply(lambda x: generator(x, max_length=80, do_sample=True)[0]["generated_text"])

# View result
print(df[["Agency Name", "Complaint Type", "Descriptor", " Resolution Description", "Dissatisfaction Reason", "Justified Reason"]])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


               Agency Name       Complaint Type        Descriptor  \
0  Department of Buildings  Adult Establishment  Zoning Violation   

                              Resolution Description  \
0  The Department of Buildings investigated this ...   

             Dissatisfaction Reason Justified Reason  
0  Agency did not correct the issue            False  


In [38]:
df['input_text']

0    Generate a detailed justified reason for this ...
Name: input_text, dtype: object

In [5]:

# Assuming your DataFrame is already loaded as df
# Filter for Department of Buildings
dept_data = data[data['Agency Name'] == 'Department of Buildings'].copy()

# Drop NA rows for relevant columns to avoid errors
dept_data.dropna(subset=['Agency Name', 'Resolution Description', 'Dissatisfaction Reason'], inplace=True)

# Function to create prompt
def create_prompt(row):
    return (
        f"Agency: {row['Agency Name']}. "
        f"Resolution: {row['Resolution Description']}. "
        f"Dissatisfaction Reason: {row['Dissatisfaction Reason']}. "
        f"Justify the issue."
    )

# Create input prompts
dept_data['input_text'] = dept_data.apply(create_prompt, axis=1)

# Take a small sample
sample_df = dept_data.sample(10, random_state=42)
input_texts = sample_df['input_text'].tolist()

# Load T5 generation pipeline
generator = pipeline("text2text-generation", model="t5-base", tokenizer="t5-base")

# Generate outputs
generated = generator(input_texts, max_length=80, do_sample=True)

# Extract the text
generated_texts = [g['generated_text'] for g in generated]

# Attach to DataFrame
sample_df['Justified Reason'] = generated_texts

# Show results
sample_df[['Agency Name', 'Resolution Description', 'Dissatisfaction Reason', 'Justified Reason']]


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes

,Agency Name,Resolution Description,Dissatisfaction Reason,Justified Reason
363634,Department of Buildings,The Department of Buildings investigated this ...,The Agency did not correct the issue.,False
82500,Department of Buildings,The Department of Buildings investigated this ...,Other,False
145484,Department of Buildings,The Department of Buildings investigated this ...,Other,False
12501,Department of Buildings,The Department of Buildings attempted to inves...,The Agency did not correct the issue.,False
315453,Department of Buildings,The Department of Buildings reviewed this comp...,The Agency did not correct the issue.,False
194077,Department of Buildings,The Department of Buildings attempted to inves...,The Agency did not correct the issue.,False
299581,Department of Buildings,The Department of Buildings investigated this ...,Other,False
157021,Department of Buildings,The Department of Buildings reviewed this comp...,The Agency did not correct the issue.,False
53721,Department of Buildings,The Department of Buildings investigated this ...,The Agency did not correct the issue.,False
85984,Department of Buildings,The Department of Buildings reviewed this comp...,The Agency did not correct the issue and this ...,False


In [13]:
# Get a sample of 5 rows from dept_data with the specified columns
sample_df = dept_data[['Agency Name', 'Complaint Type', 'Descriptor', 'Dissatisfaction Reason', 'Resolution Description']].dropna().sample(5, random_state=42)


In [14]:
def format_prompt(row):
    return (
        f"Given the following complaint:\n"
        f"Agency: {row['Agency Name']}\n"
        f"Complaint Type: {row['Complaint Type']}\n"
        f"Descriptor: {row['Descriptor']}\n"
        f"Dissatisfaction Reason: {row['Dissatisfaction Reason']}\n"
        f"Resolution Description: {row['Resolution Description']}\n"
        f"Explain why the complaint might be justified."
    )

sample_df['input_text'] = sample_df.apply(format_prompt, axis=1)


In [15]:
from transformers import pipeline

# Define generator
generator = pipeline("text2text-generation", model="t5-base", tokenizer="t5-base")

# Generate output (for small batch first)
inputs = sample_df['input_text'].tolist()
generated = generator(inputs, max_length=100, do_sample=True)

# Add generated output back to DataFrame
sample_df['Justified Reason'] = [g['generated_text'] for g in generated]


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

In [16]:
sample_df[['Agency Name', 'Complaint Type', 'Descriptor', 'Dissatisfaction Reason', 'Resolution Description', 'Justified Reason']]


,Agency Name,Complaint Type,Descriptor,Dissatisfaction Reason,Resolution Description,Justified Reason
274770,Department of Buildings,Construction In Progress,After Hours,Other,The Department of Buildings investigated this ...,True
288107,Department of Buildings,Scaffold/Sidewalk Shed,No Permit/Certificate,Other,The Department of Buildings investigated this ...,True
210403,Department of Buildings,Construction In Progress,Contrary to Plan,Other,The Department of Buildings investigated this ...,True
337349,Department of Buildings,Vent/Exhaust System,Illegal,Other,The Department of Buildings reviewed this comp...,True
157961,Department of Buildings,Boiler,Defective,Other,The Department of Buildings investigated this ...,True


In [17]:
sample_df['input_text'] = (
    "Given the following case:\n"
    "Agency: " + sample_df['Agency Name'] + "\n"
    "Complaint Type: " + sample_df['Complaint Type'] + "\n"
    "Descriptor: " + sample_df['Descriptor'] + "\n"
    "Dissatisfaction Reason: " + sample_df['Dissatisfaction Reason'] + "\n"
    "Resolution Description: " + sample_df['Resolution Description'] + "\n\n"
    "Write a detailed justification for the customer's dissatisfaction:"
)


In [18]:
generated = generator(sample_df['input_text'].tolist(), max_length=150, do_sample=True)
sample_df['Justified Reason'] = [g['generated_text'] for g in generated]


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [20]:
sample_df['Justified Reason']

274770    True
288107    True
210403    True
337349    True
157961    True
Name: Justified Reason, dtype: object

In [21]:
sample_df['input_text'] = (
    "Agency: " + sample_df['Agency Name'] + ". "
    "Complaint Type: " + sample_df['Complaint Type'] + ". "
    "Issue Description: " + sample_df['Descriptor'] + ". "
    "Dissatisfaction Reason: " + sample_df['Dissatisfaction Reason'] + ". "
    "Resolution Provided: " + sample_df['Resolution Description'] + ". "
    "Explain why the customer might still be dissatisfied:"
)


In [22]:
generated = generator(
    sample_df['input_text'].tolist(), 
    max_new_tokens=80, 
    do_sample=True, 
    top_k=50, 
    top_p=0.95,
    temperature=0.7
)
sample_df['Justified Reason'] = [g['generated_text'] for g in generated]


In [24]:
sample_df['Justified Reason']


274770    False
288107    False
210403    False
337349    False
157961    False
Name: Justified Reason, dtype: object

In [25]:
def format_input(row):
    return (
        f"Given the agency is '{row['Agency Name']}', the complaint type is '{row['Complaint Type']}', "
        f"the descriptor is '{row['Descriptor']}', and the dissatisfaction reason is '{row['Dissatisfaction Reason']}', "
        f"explain if the complaint resolution is justified or not."
    )

sample_df['input_text'] = sample_df.apply(format_input, axis=1)


In [26]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="t5-base", tokenizer="t5-base")
inputs = sample_df['input_text'].tolist()

generated = generator(inputs, max_length=100, do_sample=True)

sample_df['Justified Reason'] = [g['generated_text'] for g in generated]


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

In [27]:
sample_df[['input_text', 'Justified Reason']].head()


,input_text,Justified Reason
274770,"Given the agency is 'Department of Buildings',...",the complaint type is 'Construction In Progres...
288107,"Given the agency is 'Department of Buildings',...","is 'No Permit/Certificate', the complaint type..."
210403,"Given the agency is 'Department of Buildings',...",the complaint type is 'Construction In Progres...
337349,"Given the agency is 'Department of Buildings',...","the agency is 'Department of Buildings', the c..."
157961,"Given the agency is 'Department of Buildings',...","the agency is 'Department of Buildings', the c..."


In [28]:
# Show the full prompt and its generated justified reason
for idx, row in sample_df[['input_text', 'Justified Reason']].head(10).iterrows():
    print(f"🔹 Input Text:\n{row['input_text']}\n")
    print(f"✅ Justified Reason Generated:\n{row['Justified Reason']}\n")
    print("="*100)


🔹 Input Text:
Given the agency is 'Department of Buildings', the complaint type is 'Construction In Progress', the descriptor is 'After Hours', and the dissatisfaction reason is 'Other', explain if the complaint resolution is justified or not.

✅ Justified Reason Generated:
the complaint type is 'Construction In Progress', the descriptor is 'After Hours', and the dissatisfaction reason is 'Other', explain if the complaint resolution is justified or not.

🔹 Input Text:
Given the agency is 'Department of Buildings', the complaint type is 'Scaffold/Sidewalk Shed', the descriptor is 'No Permit/Certificate', and the dissatisfaction reason is 'Other', explain if the complaint resolution is justified or not.

✅ Justified Reason Generated:
is 'No Permit/Certificate', the complaint type is 'Scaffold/Sidewalk Shed', the descriptor is 'No Permit/Certificate', and the dissatisfaction reason is 'Other', explain if the complaint resolution is justified or not.

🔹 Input Text:
Given the agency is 'Dep

In [ ]:
from transformers import pipeline

# Use a more powerful, instruction-tuned model
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    tokenizer="google/flan-t5-large"
)

# Example input
input_text = (
    "You are an AI system that explains if a complaint resolution is justified. "
    "Agency: Department of Buildings. "
    "Complaint Type: Construction In Progress. "
    "Descriptor: After Hours. "
    "Dissatisfaction Reason: Other. "
    "Based on these, is the resolution justified or not? Give a short explanation."
)

# Generate the output
output = generator(input_text, max_length=100, do_sample=True)
print(output[0]['generated_text'])


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  50%|####9     | 1.55G/3.13G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  69%|######8   | 2.16G/3.13G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


No


In [32]:
from transformers import pipeline

# Initialize instruction-tuned model
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    tokenizer="google/flan-t5-large"
)

input_text = (
    "You are an AI system that determines if a complaint resolution is justified or not. "
    "You must base your answer on the agency, complaint type, descriptor, and dissatisfaction reason. "
    "Provide your final decision (Justified or Not Justified) followed by a brief explanation. "
    "\n\n"
    "Agency: Department of Buildings\n"
    "Complaint Type: Construction In Progress\n"
    "Descriptor: After Hours\n"
    "Dissatisfaction Reason: Other\n"
    "Resolution Description: The Department of Buildings has resolved the complaint.\n"
    "Is the resolution justified?"
)

# Generate the output
output = generator(
    input_text,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Print result
print("📝 Justified Reason:\n", output[0]['generated_text'])


Device set to use cpu


📝 Justified Reason:
 Not Justified


In [33]:
def build_prompt(row):
    return (
        "You are an AI system that determines if a complaint resolution is justified or not. "
        "You must base your answer on the agency, complaint type, descriptor, dissatisfaction reason, "
        "and resolution description. Provide your final decision (Justified or Not Justified) followed by a brief explanation.\n\n"
        f"Agency: {row['Agency Name']}\n"
        f"Complaint Type: {row['Complaint Type']}\n"
        f"Descriptor: {row['Descriptor']}\n"
        f"Dissatisfaction Reason: {row['Dissatisfaction Reason']}\n"
        f"Resolution Description: {row['Resolution Description']}\n"
        "Is the resolution justified?"
    )


In [35]:
sample_df = dept_data.sample(5, random_state=42).copy()
sample_df['input_text'] = sample_df.apply(build_prompt, axis=1)


In [36]:
from transformers import pipeline

generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    tokenizer="google/flan-t5-large"
)

# Generate outputs
generated = generator(sample_df['input_text'].tolist(), max_length=100, do_sample=True)

# Store results
sample_df['Justified Reason'] = [g['generated_text'] for g in generated]


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

In [37]:
for _, row in sample_df.iterrows():
    print("🔹 Input Prompt:\n", row['input_text'])
    print("📝 Justified Reason:\n", row['Justified Reason'])
    print("="*100)


🔹 Input Prompt:
 You are an AI system that determines if a complaint resolution is justified or not. You must base your answer on the agency, complaint type, descriptor, dissatisfaction reason, and resolution description. Provide your final decision (Justified or Not Justified) followed by a brief explanation.

Agency: Department of Buildings
Complaint Type: Certificate of Occupancy
Descriptor: Illegal Change in Use
Dissatisfaction Reason: The Agency did not correct the issue.
Resolution Description: The Department of Buildings investigated this complaint and determined that no further action was necessary.
Is the resolution justified?
📝 Justified Reason:
 Not Justified
🔹 Input Prompt:
 You are an AI system that determines if a complaint resolution is justified or not. You must base your answer on the agency, complaint type, descriptor, dissatisfaction reason, and resolution description. Provide your final decision (Justified or Not Justified) followed by a brief explanation.

Agency: 